In [1]:
import pandas as pd
import os
import re
import numpy as np
import ujson as json
import dill
import pickle
from bottleneck import push

In [3]:
data_path = 'extracts/'
with open(os.path.join(data_path, '....pkl'), 'rb') as f:
    origional= pickle.load(f)

with open(os.path.join(data_path, '....pkl'), 'rb') as f:
    interventions= pickle.load(f)

with open(os.path.join(data_path, '....pkl'), 'rb') as f:
    static_data= pickle.load(f)
with open(os.path.join(data_path, '....pkl'), 'rb') as f:
    oucomes= pickle.load(f)

Functions used to normalize and renomrmalize the data

In [ ]:
def renormalize(data, means, stds):
    renorms=[]
    for patient in data:
        r= patient*stds+ means
        renorms.append(r)
    renorm_full = np.stack(renorms, axis=0)
    return(renorm_full)
def normalize(data, mins, maxs):

    renorms=[]

    for patient in data:

        r= (patient-mins) / (np.array(maxs) - np.array(mins) + 1e-7)

        renorms.append(r)

    renorm_full = np.stack(renorms, axis=0)

    return(renorm_full)

Input for IMM and Mean imputations

In [5]:
def prepare_fills(df, means):
    # input for the IMM network (forward/backward fill then population mean if never observed)
    filled = np.flip(push(df, axis=1), axis=1)
    df2 =np.flip(push(filled, axis=1), axis = 1)
    for i in range(df.shape[2]):
        df2= np.where(np.isnan(df2), means[i], df2)
    return(df2)

def mean_fill(df, means):
    # popultaion mean for each feature imputation
    for i in range(df.shape[2]):
        df= np.where(np.isnan(df), means[i], df)
    return(df)

In [4]:
def create_individualized_missingness_mask(mask):
  np.set_printoptions(suppress=False, precision= 9)
  samples_len =mask.shape[0]
  time_steps = mask.shape[1]
  features = mask.shape[2]
  
  personalized_mask_full = np.empty(shape=[samples_len,time_steps,features])
  personalized_mask_patient = []
  personalized_mask_sample = np.ones(shape=[time_steps,features])
  for patient_mask in mask:
        num_measurments_per_feature = patient_mask.sum(axis=0)
        # for each patient mask
        tf=((num_measurments_per_feature)/time_steps)
        personalized_mask_patient.append(np.where(patient_mask == 0, tf, patient_mask))
    # stack all feature-specific patient masks tnto a 3d tensor
  personalized_mask_full = np.stack(personalized_mask_patient, axis=0)
  return(personalized_mask_full)

In [12]:
def create_age_column(df):
  bins = [15, 45,65, 300]
  labels = ['15-45', '45-65', '65+']
  df = pd.cut(df, bins, labels = labels,include_lowest = True)
  return(df)
def create_new_column_gender(row):
      if  row['Gender'] == 1:
          return 1
      else:
          return 0
def get_conditions(static_df, age_column_name, gender_column_name, interventions_3d):
        age_statics= static_df.sort_index()[age_column_name]
        age_statics = create_age_column(age_statics)
        age_group=np.array(list(age_statics))
        one_hot = pd.get_dummies(pd.DataFrame(age_group))
        one_hot=one_hot.rename(columns={ "0_15-45": "young", "0_45-65": "midage", "0_65+": "elderly"})
        one_hot.set_index(age_statics.index, inplace= True)
        static_df[gender_column_name] = static_df.apply(lambda w: create_new_column_g(w), axis=1)
        combined =pd.concat([one_hot,static_df[gender_column_name]], axis = 1)
        repeated_encoding = np.stack([combined]*interv_combined.shape[1], axis=1)
        repeated_encoding =np.concatenate([interv_combined, repeated_encoding], axis = 2)
        return(repeated_encoding)

Create Binary Mask

In [5]:
mask=~np.isnan(origional)*1

Create Individualized Missingness Mask (IMM)

In [6]:
IMM= create_individualized_missingness_mask(mask)

Create Normalized Data

In [9]:

mins = []
maxs = []

flatten= origional.reshape(origional.shape[0]*origional.shape[1], origional.shape[2])

for i in range(origional.shape[2]):

        mins.append(np.nanmin(flatten[:,i]))

        maxs.append(np.nanmax(flatten[:,i]))

normalized =normalize(origional,mins,maxs)

get means and std

In [ ]:
means = []
stds = []
flatten= normalized.reshape(origional.shape[0]*origional.shape[1], 35)
for i in range(35):
    means.append(np.nanmean(flatten[:,i]))
    stds.append(np.nanstd(flatten[:,i]))

Mean Imputation

In [10]:
mean_imputed =mean_fill(normalized, means)

Get Input for IMM (backward and forward fill, and population mean for never observed)

In [11]:
imputed=prepare_fills(normalized, means)

Last Observation Carried Forward (LOCF), 
all missing values with no prior measurments, are replaced with zero

In [8]:
LOCV =push(normalized, axis=1)
LOCV= np.where(np.isnan(LOCV), 0, LOCV)

Zero Imputation

In [10]:
zero= np.where(np.isnan(normalized), 0, normalized)

Create Conditions

In [ ]:
# inputs : static df , name of age column, name of gender column, and intervention 3d tensor
# output : 3d tensor with conditions
stat=get_conditions(static_data, "Age", "Gender", interv_combined)

In [ ]:

with open('extracts/condition.pkl', 'wb') as outfile:
    dill.dump(stat, outfile, pickle.HIGHEST_PROTOCOL) 

In [12]:

with open('extracts/zero_combined.pkl', 'wb') as outfile:
    dill.dump(zero, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/IMM_combined.pkl', 'wb') as outfile:
    dill.dump(IMM, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/mask_combined.pkl', 'wb') as outfile:
    dill.dump(mask, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/LOCV_combined.pkl', 'wb') as outfile:
    dill.dump(LOCV, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/origional_combined.pkl', 'wb') as outfile:
    dill.dump(origional, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/mins.pkl', 'wb') as outfile:
    dill.dump(mins, outfile, pickle.HIGHEST_PROTOCOL)    
with open('extracts/maxs.pkl', 'wb') as outfile:
    dill.dump(maxs, outfile, pickle.HIGHEST_PROTOCOL)
with open('extracts/mean_imputed.pkl', 'wb') as outfile:
    dill.dump(mean_imputed, outfile, pickle.HIGHEST_PROTOCOL) 

In [ ]:
with open('extracts/normalized_combined.pkl', 'wb') as outfile:
    dill.dump(normalized, outfile, pickle.HIGHEST_PROTOCOL) 

In [ ]:
with open('extracts/backforward.pkl', 'wb') as outfile:
    dill.dump(imputed, outfile, pickle.HIGHEST_PROTOCOL) 